In [20]:
import os

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from torchvision.utils import make_grid

import matplotlib.pyplot as plt

from dataset import Dataset
from tools import getDataset, print_class_distribution

import numpy as np
from sklearn.model_selection import train_test_split

In [21]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    print("MPS device not found.")
print('Device:', device)

Device: mps


In [22]:
batch_size = 128
num_workers = 4
learning_rate = 0.0001 
num_epochs = 15
image_size = 84
num_classes = 50

root_dir = os.path.join(os.getcwd(), 'datasets/miniImageNet')
dataset, label_mapping = getDataset(path=root_dir, num_classes=num_classes)


train_transforms = transforms.Compose(
        [
            transforms.RandomCrop(84, padding=8),
            transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
            transforms.GaussianBlur(3, sigma=(0.1, 2.0)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(degrees=5),
            transforms.ToTensor(),
            transforms.Normalize((0.3891, 0.3891, 0.3891), (0.2877, 0.2877, 0.2877)),
        ]
    )

transforms = transforms.Compose(
        [
            transforms.RandomCrop(84, padding=8),
            transforms.ToTensor(),
            transforms.Normalize((0.3891, 0.3891, 0.3891), (0.2877, 0.2877, 0.2877)),
        ]
    )
       
train_dataset, temp_dataset = train_test_split(dataset, test_size=0.3, random_state=42)
val_dataset, test_dataset = train_test_split(temp_dataset, test_size=0.5, random_state=42)
      
train_dataset = Dataset(dataset=train_dataset, path=root_dir, phase='train', transform=train_transforms)
val_dataset = Dataset(dataset=val_dataset, path=root_dir, phase='val', transform=transforms)
test_dataset = Dataset(dataset=test_dataset, path=root_dir, phase='test', transform=transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
validation_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [23]:
def eval(net, data_loader, criterion=nn.CrossEntropyLoss()):
    use_cuda = torch.cuda.is_available()
    if use_cuda:
        net = net.cuda()
    net.eval()
    correct = 0.0
    num_images = 0.0
    loss = 0.0
    for i_batch, (images, labels) in enumerate(data_loader):
        images, labels = images.to(device), labels.to(device)
        outs = net(images)
        loss += criterion(outs, labels).item()
        _, predicted = torch.max(outs.data, 1)
        correct += (predicted == labels).sum().item()
        num_images += len(labels)
        print('testing/evaluating -> batch: %d correct: %d numb images: %d' % (i_batch, correct, num_images) + '\r', end='')
    acc = correct / num_images
    loss /= len(data_loader)
    return acc, loss


# training function
def train(net, train_loader, valid_loader):

    criterion = nn.CrossEntropyLoss()
    # optimizer = torch.optim.SGD(params= net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0001)
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=0.0001, betas=(0.5, 0.999))
    scheduler = StepLR(optimizer, step_size=7, gamma=0.01)

    use_cuda = torch.cuda.is_available()
    if use_cuda:
        net = net.cuda()

    training_losses = []
    training_losses_epoch = []
    val_losses = []
    for epoch in range(num_epochs):
        net.train()
        correct = 0.0  # used to accumulate number of correctly recognized images
        num_images = 0.0  # used to accumulate number of images
        total_loss = 0.0

        for i_batch, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            output_train = net(images)
            loss = criterion(output_train, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            predicts = output_train.argmax(dim=1)
            correct += predicts.eq(labels).sum().item()
            num_images += len(labels)
            total_loss += loss.item()
            training_losses.append(loss.item())

            print('training -> epoch: %d, batch: %d, loss: %f' % (epoch, i_batch, loss.item()) + '\r', end='')

        print()
        acc = correct / num_images
        acc_eval, val_loss = eval(net, valid_loader)
        average_loss = total_loss / len(train_loader)
        val_losses.append(val_loss)
        training_losses_epoch.append(average_loss)
        print('\nepoch: %d, lr: %f, accuracy: %f, avg. loss: %f, valid accuracy: %f valid loss: %f\n' % (epoch, optimizer.param_groups[0]['lr'], acc, average_loss, acc_eval, val_loss))

        scheduler.step()

    return net, training_losses, training_losses_epoch,val_losses

In [24]:
from models.resnet18 import ResNet18

print(f"Hyperparameters:")
print(f"Batch Size: {batch_size}")
print(f"Learning Rate: {learning_rate}")
print(f"Number of Epochs: {num_epochs}")
print(f"Number of Workers: {num_workers}")
print(f"Number of Classes: {num_classes}\n")

# print_class_distribution(train_dataset, "Training", label_mapping)
# print_class_distribution(val_dataset, "Validation", label_mapping)
# print_class_distribution(test_dataset, "Testing", label_mapping)

model = ResNet18(num_classes=num_classes).to(device)

for param in model.parameters():
    param.requires_grad = True
    
model, training_losses, training_losses_epoch, val_losses = train(net=model, train_loader=train_loader, valid_loader=validation_loader)

acc_test, test_loss = eval(model, test_loader)
print('\naccuracy on testing data: %f' % acc_test)

plt.plot(training_losses_epoch, label='Training Loss per epoch')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.plot(training_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

Hyperparameters:
Batch Size: 128
Learning Rate: 0.0001
Number of Epochs: 15
Number of Workers: 4
Number of Classes: 50



In [ ]:
torch.save(model.state_dict(), os.path.join(os.getcwd(), f'pretrained/resnet_model_best_{round(acc_test*100, 2)}.pth'))

In [ ]:
model.eval()  

for i_batch, (images, labels) in enumerate(test_loader):
    images, labels = images.to(device), labels.to(device)
    outs = model(images)
    _, predicted = torch.max(outs.data, 1)
    print(predicted)
    print(labels)
    _, predicted = torch.max(outs.data, 1)
    correct = (predicted == labels).sum().item()
    print(correct, len(labels))
    break

tensor([16, 33, 31,  6, 42,  2, 15, 15, 48, 44, 19, 49, 39,  5, 33, 16,  7, 41,
         6, 34, 19, 42,  6, 26, 25, 11, 45, 43, 28, 28, 46, 41, 14, 49, 45, 43,
        23, 35, 30, 37, 16, 25, 39,  7, 37, 45, 43, 40,  8, 37, 30, 33, 27, 20,
        42, 49, 19,  2, 11, 29, 25, 10, 31, 29, 23, 18, 21, 21, 33, 37, 44, 15,
        15, 38, 16, 38, 24, 21,  9, 13, 24, 21, 27, 26,  8, 25, 38, 24, 21, 26,
        27, 49, 36, 41, 21, 14, 42,  0,  3,  1, 21, 34, 21, 33, 34,  3, 43, 27,
        19,  5, 31, 37, 36, 34, 16, 28, 11, 10, 20, 44,  5, 29, 42,  9, 33,  4,
        41, 46], device='mps:0')
tensor([16, 33, 15, 16, 42,  2, 26, 15,  6, 44, 19, 49, 39,  5, 26, 16, 33,  1,
         6, 34,  4, 42,  6, 26, 42, 40,  5, 43, 28, 28, 46, 41, 14, 49, 45, 43,
        23, 35, 30, 37, 44, 25, 39,  7, 37, 45, 43, 19,  8, 37, 30, 26, 27, 20,
        42, 49, 19,  2, 17, 29, 25, 22, 17, 29, 23, 18, 21, 21, 33, 37, 29, 15,
        15, 38, 33, 38, 24, 21, 30, 13, 24, 21, 27, 26,  8, 25, 38, 24, 21, 26,
       

In [ ]:
104/128

0.8125